# Final - Kaggle Competition

In [ ]:
name = input("Name :")
ID = input("student ID :")

Name :추준성
student ID :2016145028


## Prepare

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

### Data download

In [ ]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Final_exam

/content/drive/MyDrive/Final_exam


In [ ]:
# 데이터 다운로드
!wget -c --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SbXvCdS1D6FppQOR3BtPtdOi1FzVuAGY' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1SbXvCdS1D6FppQOR3BtPtdOi1FzVuAGY" -O dataset.zip

In [ ]:
# "data"폴터를 만들어서 그 안에 압축 풀기
!unzip -d ./data dataset.zip

### Data Load with augmnetation

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop((224,224)),
    transforms.ToTensor(), #이미지를 tensor자료형으로 변한
    transforms.Normalize(
                mean=torch.tensor([0.485, 0.456, 0.406]),
                std=torch.tensor([0.229, 0.224, 0.225]))]) # 정규화

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(), #이미지를 tensor자료형으로 변한
    transforms.Normalize(
                mean=torch.tensor([0.485, 0.456, 0.406]),
                std=torch.tensor([0.229, 0.224, 0.225]))]) # 정규화

# batch 크기 최적값 32
batch_size = 32

trainset = torchvision.datasets.ImageFolder(root='./data/Train', transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valset = torchvision.datasets.ImageFolder(root='./data/Val', transform=transform_test)
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)


### Set labels (변경 금지)

In [ ]:
#alphable label
from string import ascii_lowercase

alphabet_list = list(ascii_lowercase)
alphabel_label =[]
for alphabet in alphabet_list:
    for sub in alphabet_list:
        alphabel_label.append(alphabet+sub)
alphabel_label = alphabel_label[:120]

## Model

In [ ]:
# torchvision에서 models를 import
from torchvision import models

In [ ]:
# models에서 사전학습 된 efficientnet_b4를 업로드
efficientnet_b4 = models.efficientnet_b4(pretrained=True)

In [ ]:
# efficientnet_b4 모델의 fc layer의 output 클래스의 개수를 120개로 변경(본 프로젝트 데이터셋의 클래스가 120개이기 때문) 
num_ftrs = efficientnet_b4.classifier[1].in_features
efficientnet_b4.classifier[1] = nn.Linear(num_ftrs, 120)

## Training

### Training-function

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

class Trainer:
  def __init__(self, model, train_loader, opt, epoch_size=10, learning_rate=1e-2, use_cuda=True ):
    self.use_cuda = use_cuda #gpu 사용 여부
    if use_cuda :
      self.net = model.cuda() # gpu연산을 위한 model to gpu
    else :
      self.net = model
    self.train_loader = train_loader 
    self.opt = opt # optmizer 종류 설정을 위한 인자
    self.epoch_size = epoch_size # epoch
    self.learning_rate = learning_rate 
    self.validation_epoch = 2 #int(epoch_size/5) # validation 빈도
    self.criterion = nn.CrossEntropyLoss() # loss


  def train(self, valdiation_loader=None):
  
    #Optimizer
    if self.opt == "SGD":
      optimizer = optim.SGD(self.net.parameters(), lr=self.learning_rate)
    elif self.opt == "Adam":
      optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) 
    elif self.opt == "Adagrad":
      optimizer = optim.Adagrad(self.net.parameters(), lr=self.learning_rate) 

    max_test_acc = 0
    for epoch in tqdm(range(self.epoch_size)):
      self.net.train() # weight&bias를 update할 수 있는 상태로 변환

      for i, (inputs, targets) in enumerate(self.train_loader): 
        if self.use_cuda: #gpu연산
          inputs = inputs.cuda() 
          targets = targets.cuda()

        # gradient를 0으로 초기화
        optimizer.zero_grad()
        
        outputs = self.net(inputs) #forward 
        loss = self.criterion(outputs, targets) #loss계산
        loss.backward() #backward -> gradient계산
        optimizer.step() #최적화 -> weight&bias update


      if (epoch+1)%self.validation_epoch == 0:
        train_acc = self.validate(self.train_loader)
        line = 'Train Acc | Epoch {:d}/{:d} | ACC {:f}'.format((epoch+1), self.epoch_size, train_acc)
        print(line)

        if valdiation_loader != None :
          val_acc = self.validate(valdiation_loader)
          line_val = 'Val Acc | Epoch {:d}/{:d} | ACC {:f}'.format((epoch+1), self.epoch_size, val_acc)
          print(line_val)
        
        print("============================================")

  @torch.no_grad()
  def validate(self, data_loader):
    self.net.eval()
    for i, (inputs, targets) in enumerate(data_loader): 
      if self.use_cuda: #gpu연산
        inputs = inputs.cuda() 
        targets = targets.cuda()

      pred = self.net(inputs) #forward
      pred = torch.argmax(pred, dim = 1) #accuracy 측정
      pred_c = pred
      target_c = targets

      if i == 0:
        acc = (pred_c==target_c)
      else : 
        acc = torch.cat([acc, (pred_c==target_c)], axis=0)
        
    acc = acc.float().mean() * 100

    return acc.item()

  @torch.no_grad()
  def test_for_submit(self, data_loader): #kaggle 제출용 def
    ID = []
    prediction = []
    self.net.eval()
    for i, (id, image) in enumerate(data_loader): 
      if self.use_cuda: #gpu연산
        image = image.cuda() 
      
      pred = self.net(image) #forward
      pred = torch.argmax(pred, dim = 1).cpu().tolist()
      ID.extend(list(id))
      prediction.extend(pred)

    return ID, prediction


### Training and validation

In [ ]:
# 위에서 선언한 efficient_b4를 Trainer의 model 자리에 삽입(실험적으로 최적 epoch_size = 16, lr = 0.05임을 찾음)
trainer = Trainer(efficientnet_b4, train_loader, "SGD",epoch_size=16,learning_rate=0.05) #trainer 객체 선언
trainer.train(val_loader) #training and validation

  0%|          | 0/16 [00:00<?, ?it/s]

Train Acc | Epoch 2/16 | ACC 71.452866
Val Acc | Epoch 2/16 | ACC 78.012634
Train Acc | Epoch 4/16 | ACC 79.373184
Val Acc | Epoch 4/16 | ACC 85.738579
Train Acc | Epoch 6/16 | ACC 82.053772
Val Acc | Epoch 6/16 | ACC 87.342079
Train Acc | Epoch 8/16 | ACC 84.078392
Val Acc | Epoch 8/16 | ACC 88.532555
Train Acc | Epoch 10/16 | ACC 85.285065
Val Acc | Epoch 10/16 | ACC 88.678329
Train Acc | Epoch 12/16 | ACC 86.062523
Val Acc | Epoch 12/16 | ACC 89.188538
Train Acc | Epoch 14/16 | ACC 86.621323
Val Acc | Epoch 14/16 | ACC 89.723030
Train Acc | Epoch 16/16 | ACC 87.269196
Val Acc | Epoch 16/16 | ACC 89.382896


## Test and Submit (변경 금지)

#### Test Loader

In [ ]:
import os

import cv2
import random
from PIL import Image
from torch.utils import data

class TestLoader(data.Dataset):
    def __init__(self, root, transform=None):
        super(TestLoader, self).__init__()

        self.root = root
        self.transform = transform
        self.test_root = os.path.join(self.root, "test")
        self.files = []
        self.id = []
        for i, (root, dirs, files) in enumerate(os.walk(self.test_root)):
          files.sort()
          for file in files:
            self.files.append(os.path.join(self.test_root,file))
            self.id.append(file.split('.')[0])


    def __getitem__(self, index):
        image_path = self.files[index]

        image = Image.open(image_path).convert('RGB')
        image_id = self.id[index]
        if self.transform:
            image = self.transform(image)

        return image_id, image

    def __len__(self):
        return len(self.files)


In [ ]:
testset = TestLoader(root='./data/Test', transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=1) #Shuffle=False 절대 변경 금지!!!

### Test

In [ ]:
ID, prediction = trainer.test_for_submit(test_loader)

### Kaggle 제출용

In [ ]:
import csv
#csv파일 생성
with open('submit_efficientnet_baseline.csv', mode='w') as pred_file:
  pred_writer = csv.writer(pred_file, delimiter=',')
  pred_writer.writerow(['Id', 'Category'])
  for i in range(len(prediction)): 
    pred_writer.writerow([ID[i], alphabel_label[prediction[i]]])
  print("end ID : ",i)


end ID :  4115
